In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout, WeakParabolicBottom, ConnectedGraphRMS
rt   = rtsvg.RACETrack()

g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}
_relates_ = [('fm','to')]
_colors_  = {n:'red' for n in _to_fix_}
_psl_     = PolarsSpringLayout(g)
pos       = {}
_results_ = _psl_.results()
print(f'original results ... {ConnectedGraphRMS(g, _results_).rms(1.0, 1.0)=}')
for x in _results_: pos[x] = (_results_[x][0]*0.5, _results_[x][1]*2.0) # x_scale is 0.5 and y_scale is 2.0
print(f'after applied scaling ... {ConnectedGraphRMS(g, pos).rms(2.0, 0.5)=}')
print(f'if left unscaled ... {ConnectedGraphRMS(g, pos).rms(1.0, 1.0)=}')
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
print(f'new_pos (after mod) ... {ConnectedGraphRMS(g, new_pos).rms(1.0, 1.0)=}')
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
_tiles_ = []
def makeTile():
    _lu_ = {'x':[], 'y':[], 'group':[]}
    x = 0.5
    while x < 20.0:
        _rms_ = _psl_.rootMeanSquareError(x, y_scale)
        _lu_['x'].append(x), _lu_['y'].append(_rms_), _lu_['group'].append('x')
        x += 0.5
    y = 0.5
    while y < 20.0:
        _rms_ = _psl_.rootMeanSquareError(x_scale, y)
        _lu_['x'].append(y), _lu_['y'].append(_rms_), _lu_['group'].append('y')
        y += 0.5
    return rt.xy(pl.DataFrame(_lu_), x_field='x', y_field='y', line_groupby_field='group', color_by='group', dot_size=None)

x_scale, y_scale = 1.0, 1.0 # initial guesses for the scaling factors
for i in range(10):
    def _xfn_(x): return _psl_.rootMeanSquareError(x, y_scale)
    _wpb_x_ = WeakParabolicBottom(_xfn_)
    x_scale = _wpb_x_.xys[-1][0]
    def _yfn_(y): return _psl_.rootMeanSquareError(x_scale, y)
    _wpb_y_ = WeakParabolicBottom(_yfn_)
    y_scale = _wpb_y_.xys[-1][0]
    print(f'({x_scale:9.3f} {y_scale:9.3f})')
    _tiles_.append(makeTile())
scaled_pos = {}
for x in new_pos:
    scaled_pos[x] = (new_pos[x][0] * x_scale, new_pos[x][1] * y_scale)
params = {'w':384, 'h':384, 'node_color':_colors_}
scaled_pos_with_springs = copy.deepcopy(scaled_pos)
scaled_pos_with_springs = PolarsSpringLayout(g, pos=scaled_pos_with_springs, static_nodes=set(g.nodes()) - _to_fix_).results()
rt.tile([rt.link(df, _relates_, scaled_pos,              **params),
         rt.link(df, _relates_, scaled_pos_with_springs, **params)], spacer=10)

In [ ]:
#rt.tile(_tiles_)

In [ ]:
conn_rms = ConnectedGraphRMS(g, new_pos)

x_scale, y_scale = 1.0, 1.0 # initial guesses for the scaling factors
_scales_         = []
for i in range(10):
    def _xfn_(x): return conn_rms.rms(x, y_scale)
    _wpb_x_ = WeakParabolicBottom(_xfn_)
    x_scale = _wpb_x_.xys[-1][0]
    def _yfn_(y): return conn_rms.rms(x_scale, y)
    _wpb_y_ = WeakParabolicBottom(_yfn_)
    y_scale = _wpb_y_.xys[-1][0]
    print(f'({x_scale:9.3f} @ {_wpb_x_.xys[-1][1]} | {y_scale:9.3f} @ {_wpb_y_.xys[-1][1]})')
    _tiles_.append(makeTile())
    _scales_.append((x_scale, y_scale))
    if len(_scales_) > 1 and \
       round(_scales_[-1][0],3) == round(_scales_[-2][0],3) and \
       round(_scales_[-1][1],3) == round(_scales_[-2][1],3): break
scaled_pos = {}
for x in new_pos: scaled_pos[x] = (new_pos[x][0] * x_scale, new_pos[x][1] * y_scale)
params = {'w':384, 'h':384, 'node_color':_colors_}
scaled_pos_with_springs = copy.deepcopy(scaled_pos)
scaled_pos_with_springs = PolarsSpringLayout(g, pos=scaled_pos_with_springs, static_nodes=set(g.nodes()) - _to_fix_).results()
rt.tile([rt.link(df, _relates_, scaled_pos,              **params),
         rt.link(df, _relates_, scaled_pos_with_springs, **params)], spacer=10)


In [ ]:
df_single_loc    = pl.DataFrame({'fm':['a','b'], 'to':['b','a']})
pos_single_loc   = {'a':(0.5, 0.5), 'b':(0.5, 0.5)}
g_single_loc     = rt.createNetworkXGraph(df_single_loc, relationships=[('fm','to')])
_psl_single_loc_ = PolarsSpringLayout(g_single_loc, pos=pos_single_loc)
_psl_single_loc_.results()

In [ ]:
_rms_min_, _at_x_, _at_y_ = None, None, None
conn_rms = ConnectedGraphRMS(g, new_pos)
x = 0.1
while x < 3.0:
    y = 0.1
    while y < 3.0:
        _rms_ = conn_rms.rms(x,y)
        if _rms_min_ is None: _rms_min_, _at_x_, _at_y_ = _rms_, x, y
        if _rms_ < _rms_min_: _rms_min_, _at_x_, _at_y_ = _rms_, x, y
        y += 0.1
    x += 0.1
_rms_min_, _at_x_, _at_y_